In [1]:
from one.api import ONE
one = ONE()
eid = '4ecb5d24-f5cc-402c-be28-9d0f7cb14b3a'
passive_times = one.load_dataset(eid, '*passivePeriods*', collection='alf')
SP_times = passive_times['spontaneousActivity']

In [2]:
import numpy as np
# Find first probe insertion for session
pid = one.alyx.rest('insertions', 'list', session=eid)[0]['id']

from brainbox.io.one import SpikeSortingLoader
from iblatlas.atlas import AllenAtlas
import numpy as np
ba = AllenAtlas()

# Load in spikesorting
sl = SpikeSortingLoader(pid=pid, one=one, atlas=ba)
spikes, clusters, channels = sl.load_spike_sorting()
clusters = sl.merge_clusters(spikes, clusters, channels)

# Find spike times during spontaneous activity
SP_idx = np.bitwise_and(spikes['times'] >= SP_times[0], spikes['times'] <= SP_times[1])

# Count the number of clusters during SP time period and compute firing rate
from brainbox.population.decode import get_spike_counts_in_bins
counts, cluster_ids = get_spike_counts_in_bins(spikes['times'][SP_idx], spikes['clusters'][SP_idx],
                                               np.c_[SP_times[0], SP_times[1]])
fr = counts / (SP_times[1] - SP_times[0])

In [4]:
print(np.unique(clusters['acronym']))

['BST' 'HY' 'LPO' 'LSr' 'LSv' 'NDB' 'PS' 'STR' 'aco' 'ccb']
